In [1]:
import pandas as pd
import numpy as np
import requests
import json
import time
from bs4 import BeautifulSoup

In [2]:
url = 'https://api.yelp.com/v3/businesses/search'
response = requests.get(url)

f = open("./apikey.txt", "r")
MY_API_KEY = (f.read())

In [3]:
def yelp_data(search_term, zipcode, limit, url, api_key):
    headers = {'Authorization': 'Bearer {}'.format(api_key),}

    url_params = {
                'term': search_term.replace(' ', '+'),
                'location': zipcode,
                'limit': limit}
    a = requests.get(url, headers = headers, params= url_params)
    b = a.text
    df = json.loads(b)
    return pd.DataFrame(df['businesses'])

In [4]:
def yelp_dataframe(search_term, list_of_zipcodes, limit, url, api_key):
    main = []
    maindf = pd.DataFrame()
    for i in range(0, len(list_of_zipcodes)):
        main.append(yelp_data(search_term, list_of_zipcodes[i], limit, url, api_key))
        time.sleep(2)
    for i in main:
        maindf = pd.concat([i, maindf], axis = 0)
    return maindf

In [5]:
def closed_dataframe(list_of_restaurants, list_of_zipcodes, url, api_key):
    ylist = []
    gdataframe = pd.DataFrame()
    for i in range(0, len(list_of_restaurants)):
        ylist.append(yelp_data(list_of_restaurants[i], list_of_zipcodes[i],             1, url, api_key))
    for i in ylist:
        gdataframe = pd.concat([i, gdataframe], axis = 0)
    return gdataframe

In [6]:
def get_features(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    l = soup.find_all(class_= "lemon--span__373c0__3997G text__373c0__2Kxyz text-color--normal__373c0__3xep9 text-align--left__373c0__2XGa- text-weight--semibold__373c0__2l0fe text-size--large__373c0__3t60B")
    v = []
    for i in l:
        v.append(i.text.strip())
    return v

In [7]:
def outdoor_seating(array_of_lists):
    r = []
    for i in array_of_lists:
        if 'Outdoor seating' in i:
            r.append(1)
        else:
            r.append(0)
    return r

In [8]:
def outdoor_seating_column(alias_column):
    w = []
    for i in list(alias_column):
        url = f'https://www.yelp.com/biz/{i}'
        w.append(get_features(url))
    return outdoor_seating(w)

In [9]:
closeddf = pd.read_excel('Closed restaurant zip codes.xlsx')

In [10]:
closedlist = closeddf.Restaurant.unique()

In [11]:
ziplist = list(closeddf.zip_code)

In [12]:
opendf = yelp_dataframe('Restaurants', ziplist, 35, url, MY_API_KEY)

In [41]:
opendf.drop_duplicates(subset = ['alias'], inplace = True)

In [42]:
opendf['permanently closed'] = opendf.id.apply(lambda x: 0 if x != 0 else x)

In [44]:
opendf['outdoor_seating'] = outdoor_seating_column(opendf.alias)

In [47]:
opendf.to_csv('openrest.csv')

In [ ]:
e = closed_dataframe(closedlist, ziplist, url, MY_API_KEY)

In [ ]:
e['permanently closed'] = e.index +1

In [ ]:
e['outdoor_seating'] = outdoor_seating_column(e.alias)

In [ ]:
e.to_csv('closed.csv')